# Exercises - Genetic Algorithms

In [1]:
from math import log2, ceil

In [44]:
from random import randint,random,sample

In [3]:
from statistics import mean

In [50]:
sample([2,3,52,5],4)

[5, 2, 3, 52]

In [4]:
def f(x1,x2):
    if not(-4.5 <= x1 <= 4.5 and -4.5 <= x2 <= 4.5):
        return 'bad input: out of domain'
    return (1.5-x1+x1*x2)**2 + (2.25-x1+x1*(x2**2))**2 + (2.625-x1+x1*(x2**3))**2

El mínimo global está en x = (3, 0.5) y f(x) = 0. \*global minimum

In [5]:
f(3,0.5), f(-4.5,-4.5)

(0.0, 181853.61328125)

## 1.
Implementar en el lenguaje de su preferencia, el AG que se diseñó en el ejemplo anterior para resolver la función de Beale.

In [6]:
int('0111010000100011010101011001', 2)

121779545

In [7]:
def bits_req(x_bounds):
    return [ceil(log2((x[1]-(x[0]))*(10**3))) for x in x_bounds]

In [8]:
bits_req([(-4.5,4.5),(-4.5,4.5)])

[14, 14]

In [9]:
def real_from_bits(bit_string, n_variable_bits, n_variable_bounds):
    reals = []
    index = 0
    for n_bits, bound in zip(n_variable_bits,n_variable_bounds):
        bits = bit_string[index:index+n_bits]
        index = n_bits
        d = int(bits, 2)
        r = bound[0] + ((bound[1]-bound[0])*d) / ((2**n_bits)-1)
        reals.append(r)
    return reals

In [10]:
var_bounds = [(-4.5,4.5),(-4.5,4.5)]
n_variable_bits = bits_req(var_bounds)
n_variable_bits

[14, 14]

In [11]:
real_from_bits('11011000000000'+'10001110000000', n_variable_bits, var_bounds)

[3.0942135140084233, 0.49249221754257455]

In [12]:
real_from_bits('01110100001000'+'11010101011001', n_variable_bits, var_bounds)

[-0.41723127632301793, 3.0024720747115916]

In [13]:
example_individual = ('1111111111111111111111011001', n_variable_bits, var_bounds)
example_individual

('1111111111111111111111011001', [14, 14], [(-4.5, 4.5), (-4.5, 4.5)])

In [14]:
def Beale_fitness(bit_string, n_variable_bits_req, n_variable_bounds): # a low value is a bad fitness
    return 1/(f(*real_from_bits(bit_string, n_variable_bits_req, n_variable_bounds))+1)

In [15]:
Beale_fitness(*example_individual)

5.880385373883209e-06

In [16]:
Beale_fitness('11011000000000'+'10001110000000', n_variable_bits, var_bounds)

0.9768962222434674

In [17]:
def random_bin_string(bits):
    string = ''
    for i in range(bits):
        string += str(randint(0,1))
    return string

In [18]:
random_bin_string(4)

'1100'

In [19]:
def random_solution(n_variable_bits_req, n_variable_bounds=False):
    individual = ''
    for bits in n_variable_bits_req:
        x_bits = random_bin_string(bits)
        individual += (x_bits)
    if n_variable_bounds:
        return individual, Beale_fitness(individual, n_variable_bits_req, n_variable_bounds)
    return individual

In [20]:
b_required = bits_req([(2,3),(5,6),(-50,11)])
rand_sol = random_solution(b_required)

b_required, rand_sol, sum(b_required), len(rand_sol)

([10, 10, 16], '000101111110011010000010010101100100', 36, 36)

In [21]:
random_solution(bits_req([(-4.5,4.5),(-4.5,4.5)]))#,n_variable_bounds=[(-4.5,4.5),(-4.5,4.5)])

'0101100010010000011010000010'

In [22]:
random_solution(bits_req([(-4.5,4.5),(-4.5,4.5)]),n_variable_bounds=[(-4.5,4.5),(-4.5,4.5)])

('1111110001011001110011000010', 0.03246352578928539)

In [23]:
def universal_stochastic_selection(population, n_variable_bits_req, n_variable_bounds):
    fitnesses = [x[1] for x in population]
    strings = [x[0] for x in population]
    fitness_mean = mean(fitnesses)
    Ei_s = [fit/fitness_mean for fit in fitnesses]
    r = random()
    parents = []
    sum_ = 0
    for individual,Ei,fit in zip(strings,Ei_s,fitnesses):
        sum_ += Ei
        while sum_ > r:
            parents.append((individual,fit))
            r += 1
    return parents

In [24]:
universal_stochastic_selection([('100010110011110110011000011100110000', 0.15029431701487947),
                                ('110000100011001001100110010010011010', 0.011398346494599433),
                                ('101110111100100000001000011010001010', 2.9777439040564625e-05),
                                ('110101000010111111010100100101110110', 0.008058140115587174),
                                ('001101101111011010101101010101110010', 0.00025309048960844106)],  
                               bits_req([(-4.5, 4.5),(-4.5, 4.5)]),
                               [(-4.5, 4.5),(-4.5, 4.5)])

[('100010110011110110011000011100110000', 0.15029431701487947),
 ('100010110011110110011000011100110000', 0.15029431701487947),
 ('100010110011110110011000011100110000', 0.15029431701487947),
 ('100010110011110110011000011100110000', 0.15029431701487947),
 ('110000100011001001100110010010011010', 0.011398346494599433)]

In [25]:
def k_point_crossover(k_points, p1, p2):
    o1, o2 = '', ''
    index_closer = 0
    for i,point in enumerate(k_points+[len(p1)]):
        if i%2 == 0:
            o1 += p1[index_closer:point]
            o2 += p2[index_closer:point]
        else:
            o1 += p2[index_closer:point]
            o2 += p1[index_closer:point]
        index_closer = point
    return o1,o2

In [26]:
k_point_crossover([2,17],'1000101100hola0110011000011100110000',
 '11010100001011adios10100100101110110')

('10010100001011adi0011000011100110000',
 '1100101100hola011os10100100101110110')

In [27]:
def grouped(iterable, n):
    "s -> (s0,s1,s2,...sn-1), (sn,sn+1,sn+2,...s2n-1), (s2n,s2n+1,s2n+2,...s3n-1), ..."
    return zip(*[iter(iterable)]*n)

In [28]:
for i,j in grouped([3,2,7,4,10,9,0,14],2):
    print(i,j)

3 2
7 4
10 9
0 14


In [29]:
def mutate(individual, pm):
    mutated = ''
    for i in individual:
        if random() < pm:
            if i == '0':
                mutated += '1'
            else:
                mutated += '0'
        else:
            mutated += i
    return mutated        

In [30]:
mutate('1111111111',.1)

'1111111111'

In [31]:
def mutate_pop(population, pm, n_variable_bits_req, n_variable_bounds):
    mutated_pop = []
    length = len(population[0])
    for i in range(len(population)):
        mutated_ind = mutate(population[i][0],pm)
        mutated_pop.append((mutated_ind, Beale_fitness(mutated_ind, n_variable_bits_req, n_variable_bounds)))     
    return mutated_pop

In [32]:
mutate_pop([('11111111111111111',.4),('00000000000000000',.1)],.1,
          bits_req([(-4.5, 4.5),(-4.5, 4.5)]),
                               [(-4.5, 4.5),(-4.5, 4.5)])

[('11111111111011111', 5.647605878400432e-06),
 ('00000010000000000', 5.673925993036313e-06)]

In [99]:
def GA(μ, gens, pc, pm, n, n_variable_bounds):
    n_variable_bits_req = bits_req(n_variable_bounds)
    total_bits = sum(n_variable_bits_req)
    population = [random_solution(n_variable_bits_req,n_variable_bounds) for i in range(μ)]
    for gen in range(gens):
        parents = universal_stochastic_selection(population, n_variable_bits_req, n_variable_bounds)
        offspring = []
        for p1,p2 in grouped(parents,2):
            offs = []
            if random() < pc:
                cross_points = sorted(sample(list(range(total_bits)),2))
                o1,o2 = k_point_crossover(cross_points,p1[0],p2[0])
                o1 = (o1, Beale_fitness(o1, n_variable_bits_req, n_variable_bounds))
                o2 = (o2, Beale_fitness(o2, n_variable_bits_req, n_variable_bounds))
                offs = [o1,o2]
            else:
                offs = [p1,p2]
            offspring += offs
        
        offspring = mutate_pop(offspring,pm,n_variable_bits_req,n_variable_bounds)
        offspring.sort(key=lambda x:x[1], reverse=True)
        parents.sort(key=lambda x:x[1], reverse=True)
        
        if parents[0][1] > offspring[0][1]:
            offspring[-1] = parents[0]
            offspring.sort(key=lambda x:x[1], reverse=True)
        population = offspring
    population.sort(key=lambda x:x[1], reverse=True)
    return population[0]

In [108]:
GA(100, 1, 0.9, 0.001, 0,[(-4.5,4.5),(-4.5,4.5)])

('1111011101101110010100010010', 0.8885074394867123)

In [109]:
GA(100, 2000, 0.9, 0.001, 0,[(-4.5,4.5),(-4.5,4.5)])

('1101010101010110001110001110', 0.9999996826798102)

In [110]:
real_from_bits('1101010101010110001110001110', n_variable_bits, [(-4.5,4.5),(-4.5,4.5)])

[3.0002746749679545, 0.500183116645303]